In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

**keys**: electrostatics, homogeneous Dirichlet bvp, double layer potential $K$ and single layer potential $V$

Dirichlet Laplace Direct Method 
==========================

Consider the Dirichlet boundary value problem 

$$ \left\{ \begin{array}{rcl l} \Delta u &=& 0, \quad &\Omega \subset \mathbb R^3\,,\\ \gamma_0 u&=& m, \quad &\Gamma = \partial \Omega\,.\end{array} \right. $$ 

Choosing a direct ansatz the solution $u\in H^1(\Omega)$ is given by 

$$ u(x) = \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, j(y)\, \mathrm{d}s_y}_{\displaystyle{ \mathrm{ LaplaceSL}(j) }} - \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{\langle n(y) , x-y\rangle }{\| x-y\|^3} } \, m(y)\, \mathrm{d}s_y}_{\displaystyle{ \mathrm{LaplaceDL}(m) }}\,,$$ 

where $\mathrm m$ solves the boundary integral equation 

$$ \mathrm V = \left( \frac12 \mathrm M + \mathrm K\right) \mathrm m\quad \text{on} \; \Gamma\,.$$


Define the geometry $\Omega \subset \mathbb R^3$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.2)).Curve(4)

Create the finite element spaces for $H^{-\frac12}(\Gamma)$ and $H^{\frac12}(\Gamma)$ according to the given mesh:  

In [ ]:
fesL2 = SurfaceL2(mesh, order=3, dual_mapping=True)
u,v = fesL2.TnT()
fesH1 = H1(mesh, order=4)
uH1,vH1 = fesH1.TnT()
print ("ndofL2 = ", fesL2.ndof, "ndof H1 = ", fesH1.ndof)

Compute the interpolation of exact Dirichlet data $u_0$ in finite element space:

In [ ]:
uexa = 1/ sqrt( (x-1)**2 + (y-1)**2 + (z-1)**2 )
m = GridFunction(fesH1)
m.Interpolate (uexa)
Draw (m, mesh, draw_vol=False, order=3)

**Boundary Integral Equation** 

We carefully apply the Dirichlet trace on the ansatz for $u$, we get a boundary integral equation for the Neumann data $j \in H^{-\frac12}(\Gamma)$ which can be read as follows: determine $j$ such that 

$$\begin{array}{r c ccccc} \forall \, v\in H^{-\frac12}(\Gamma): && \displaystyle \int\limits_\Gamma \gamma_0 \left(\mathrm{SL}(j)\right) \cdot v \, \mathrm d s &=& \displaystyle \int\limits_\Gamma m \cdot v \, \mathrm d s &+& \displaystyle \int\limits_\Gamma \gamma_0 \left(\mathrm{DL}(m)\right)\cdot v \, \mathrm d s \\[2ex] & \Rightarrow & \mathrm{V}\,\mathrm{j} &=& \displaystyle \frac12\,\mathrm{M} \, \mathrm m &+&  \mathrm K\, \mathrm m \,. \end{array} $$ 

The discretisation of the above variational formulation is the boundary element method and it leads to a system of linear equations, ie 

$$ \mathrm{V} \, \mathrm{j} = \left( \frac12 \,\mathrm{M} + \mathrm{K} \right) \, \mathrm{m}\,, $$ 
where the linear operators are as follows
- $\mathrm{V}$ is the single layer operator. $\mathrm V$ is regular and symmetric.
- $\mathrm{M}$ is a mass matrix.
- $\mathrm{K}$ is the double layer operator. 

**Details**
  
For any $v_i, u_j\in H^{-\frac12}(\Gamma)$ and $w_j \in H^{\frac12}(\Gamma)$ the layer potential operators are implemented as follows 

$$ \begin{array}{rcl cl} V_{ij} &=& \langle v_i, \gamma_0 \mathrm{SL}(u_j) \rangle_{-\frac12} &=& \displaystyle \frac{1}{4\pi}\, \displaystyle \int\limits_\Gamma \int\limits_\Gamma \frac{1}{\|x-y\|} \, u_j(x) \, v_i(y) \, \mathrm{d} s_{y} \, \mathrm{d} s_x\,, \\[2ex] K_{ij} &=& \langle v_i, \gamma_0 \mathrm{DL}(w_j) \rangle_{-\frac12} &=& \displaystyle \frac{1}{4\pi} \int\limits_\Gamma\int\limits_\Gamma n_y\cdot \nabla_y \frac{1}{\|x-y\|} \, w_j(x) \, v_i(y) \, \mathrm{d} s_{y} \, \mathrm{d} s_x \end{array}$$


You may assemble $V$ and $K$ it by

```
V = LaplaceSL(u*ds)*v*ds
K = LaplaceDL(w*ds)*v*ds
```

We generate the linear operators and solve for the Neumann data $j$ using an iterative solver:

In [ ]:
j = GridFunction(fesL2)
pre = BilinearForm(u*v*ds, diagonal=True).Assemble().mat.Inverse()
with TaskManager():
    V = LaplaceSL(u*ds)*v*ds
    K = LaplaceDL(uH1*ds)*v*ds
    M = BilinearForm(uH1*v*ds(bonus_intorder=3)).Assemble()
    
    rhs = ( (0.5 * M.mat + K.mat)*m.vec).Evaluate()
    CG(mat = V.mat, pre=pre, rhs = rhs, sol=j.vec, tol=1e-8, maxsteps=50, initialize=False, printrates=True)

Draw (j, mesh, draw_vol=False, order=3);

Let's have a look at the exact Neumann data and compute the error of the numerical solution:

In [ ]:
graduexa = CF( (uexa.Diff(x), uexa.Diff(y), uexa.Diff(z)) )
n = specialcf.normal(3)
jexa = graduexa*n
Draw (jexa, mesh, draw_vol=False, order=3);
print ("L2-error =", sqrt (Integrate ( (jexa-j)**2, mesh, BND)))

**Project repformula in H1**

Evaluate the solution on a screen inside $\Omega$ 

In [ ]:
screen = WorkPlane(Axes( (0,0,0), Z, X)).RectangleC(0.5,0.5).Face()
screen.faces.name="screen"
vismesh = screen.GenerateMesh(maxh=0.05)

In [ ]:
LSPotential = LaplaceSL( u*ds )
LDPotential = LaplaceDL( uH1*ds )

In [ ]:
repformula = LSPotential(j) - LDPotential(m)
fes_screen = H1(vismesh, order=5)
gf_screen = GridFunction(fes_screen)
with TaskManager():
    gf_screen.Set(repformula, definedon=vismesh.Boundaries("screen"))
Draw(gf_screen)
Draw(repformula - uexa, vismesh)

In [ ]:
type(LSPotential)